<div>
<table style="width: 100%">
	<tr>
		<td>
		<table style="width: 100%">
			<tr>
                <td ><center><font size="5"><b>Module 49</b></font><center>
                <center><font size="6">Digital Innovations for Water Challenges</font><center></td>
			</tr>
			<tr>
                <td><center><font size="14">Notebook 2.f</font><center></td>
			</tr>
			<tr>
                <td><center><font size="6"><b>Map Algebra</b></font><center></td>
			</tr>
		</table>
		</td>
		<td><center><img src='images/ihe-delft-institute_unesco_fc-lr.jpg'></img></td>
	</tr>
</table>
</div>

# Table of contents
1. [Learning objectives](#learningobs)
2. [Introduction](#introduction)
3. [Preparation](#preparation)
4. [Condition 1: Wells within 150 Meters of Houses or Roads](#condition_1)
5. [Condition 2: No Industry, Mine, or Landfill within 300 Meters from Wells](#condition_2)
6. [Condition 3: Wells Less than 40 Meters Deep](#condition_3)
7. [Combine the Three Conditions](#combine)
8. [What is the advantage of scripting map algebra?](#conclusion)

# 1. Learning objectives<a name="learningobs"></a>
After this lesson you will be able to:

* apply map algebra for raster analysis
* distinguish Boolean, discrete, and continuous rasters
* make legends for Boolean, discrete, and continuous maps
* understand the use of Nodata
* use logical operators
* calculate distances from rasters
* reclassify rasters

# 2. Introduction<a name="introduction"></a>
With map algebra we can do calculations with raster layers. This is useful for spatial analysis. For example, when we need to evaluate different criteria to find suitable or unsuitable locations we can use map algebra.

### Case study
In this lesson we'll address the following case.
The municipality of the (imaginary) oasis Aïn Kju Dzjis has hired you to analyse which wells are unsuitable for its inhabitants based on the following conditions:

_Condition 1:_
The wells should be within 150 meters of houses or roads.

_Condition 2:_
No industry, mine, or landfill within 300 meters of the wells.

_Condition 3:_
The wells should be less than 40 meters deep.

You will use map algebra to perform the required analysis.

This lesson will follow this workflow:

![](Images/Flowchart.PNG)


### Software and data

The exercises use the PCRaster Python software. For installation instructions click [here](https://pcraster.geo.uu.nl/pcraster/latest/documentation/pcraster_project/install.html).

The raster layers for this lesson can be found in the <code>2j_Data</code> folder.

# 3. Preparation<a name="preparation"></a>
## 3.1 Introduction
For this task you are provided with the following raster layers: `buildg.tif`, `roads.tif`, `dtm.tif`, and `gwlevel.tif`. You can find these files in the `2j_Data` folder.

To work with PCRaster we need to convert the files to the PCRaster format. You can do that from the command line using the GDAL commands. Here we'll do the conversion with the GDAL Python library.

## 3.2 Convert rasters with GDAL

Let's convert the `buildg.tif` file.
First we import `gdal` and `gdalconst` from the `osgeo` library:

In [ ]:
from osgeo import gdal, gdalconst

Then we read the GeoTIFF from our `2j_Data` folder.

In [ ]:
src_ds = gdal.Open( "2j_Data/buildg.tif" )

Next we can use the Python equivalent of [`gdal_translate`](https://gdal.org/programs/gdal_translate.html#gdal-translate): `Translate`. We need to use the name of the format. See [GDAL Raster drivers (short name)](https://gdal.org/drivers/raster/index.html) for formats. In our case the output format is `PCRaster`.

We also need to define the data type of the output.

PCRaster uses data typing of the data in the database: each map has one of the six data types used attached to it. These data types help you and PCRaster to structure the data. See the table below.<br>

| data type | description of attributes | domain | example |
|:-------:|:-------:|:-------:|:-------:|
| boolean | boolean | 0 (FALSE),<br> 1 (TRUE) | suitable/unsuitable, visible/non visible |
| nominal |	classified, no order | whole values | soil classes |
| ordinal | classified, order | whole values | succession stages, income groups |
| scalar | continuous, linear | real values | temperature, concentration |
| directional | continuous, directional | 0 to 360 degrees | aspect |
| ldd | direction to neighbour cell | codes of directions | drainage networks |

In the arguments of `Translate` you need to define the `outputType` and `metadataOptions` based on the table below. Note that we can only convert boolean, nominal and scalar rasters. The ldd and directional rasters are created within PCRaster. For scalar we can use Float 32 or Float 64, depending on the desired precision.

| data type | `outputType` | `metadataOptions` |
|:-------:|:-------:|:-------:|
| boolean | `gdalconst.GDT_Byte` | `'VS_BOOLEAN'`|
| nominal | `gdalconst.GDT_Int32` | `'VS_NOMINAL'`|
| scalar | `gdalconst.GDT_Float32` | `'VS_SCALAR'`|
| scalar | `gdalconst.GDT_Float64` | `'VS_SCALAR'`|

`buildg.tif` is a nominal raster. It contains classes. Therefore the code we'll use for `Translate` is:

In [ ]:
dst_ds = gdal.Translate('2j_Data/buildg.map', src_ds, format='PCRaster', \
                        outputType=gdalconst.GDT_Int32, metadataOptions='VS_NOMINAL')

And we properly close the datasets to flush to disk:

In [ ]:
dst_ds = None
src_ds = None

Now you can find the file `buildg.map` on your hard disk. Check if that's the case if you're running this notebook locally.

Let's apply the same code to the other files `dtm.tif`, `roads.tif` and `gwlevel.tif`. In this case it's easier to write it as a function.

In [ ]:
#Import gdal
from osgeo import gdal

def ConvertToPCRaster(src_filename,dst_filename,ot,VS):
    #Open existing dataset
    src_ds = gdal.Open(src_filename)
    
    #GDAL Translate
    dst_ds = gdal.Translate(dst_filename, src_ds, format='PCRaster', outputType=ot, metadataOptions=VS)
    
    #Properly close the datasets to flush to disk
    dst_ds = None
    src_ds = None
    
ConvertToPCRaster("2j_Data/buildg.tif","data/buildg.map",gdalconst.GDT_Int32,"VS_NOMINAL")
ConvertToPCRaster("2j_Data/roads.tif","data/roads.map",gdalconst.GDT_Int32,"VS_NOMINAL")
ConvertToPCRaster("2j_Data/gwlevel.tif","data/gwlevel.map",gdalconst.GDT_Float32,"VS_SCALAR")
ConvertToPCRaster("2j_Data/dtm.tif","data/dtm.map",gdalconst.GDT_Float32,"VS_SCALAR")

## 3.3 Inspecting the data
For map algebra the properties of all raster layers used in calculations need to be the same.
Let's find out if they have the same number of rows and columns, coordinates and pixel size.

Let's open a PCRaster file with GDAL.

In [ ]:
RasterLayer = gdal.Open("2j_Data/dtm.map")

To get the numbers of rows and columns we can use the GDAL functions `RasterXSize` and `RasterYSize`.
`RasterLayer.GetDescription()` gives the relative path of the layer.

In [ ]:
Path = RasterLayer.GetDescription()
NrColumns = RasterLayer.RasterXSize
NrRows = RasterLayer.RasterYSize
print('{} has {} columns and {} rows'.format(Path,NrColumns,NrRows ))

To know the coordinates and pixel size we can use `GetGeoTransform` from GDAL.
It returns a tuple with the following information:
`RasterLayer.GetGeoTransform[0]`: Top left X coordinate

`RasterLayer.GetGeoTransform[1]`: West-East pixel resolution

`RasterLayer.GetGeoTransform[2]`: Rotation, 0 for "north up"

`RasterLayer.GetGeoTransform[3]`: Top left Y coordinate

`RasterLayer.GetGeoTransform[4]`: Rotation, 0 for "north up"

`RasterLayer.GetGeoTransform[5]`: North-South pixel resolution (negative)

In [ ]:
RasterLayer.GetGeoTransform()

Let's present that more readable.

In [ ]:
print("Origin = ({}, {})".format(RasterLayer.GetGeoTransform()[0], RasterLayer.GetGeoTransform()[3]))
print("Pixel Size = ({}, {})".format(RasterLayer.GetGeoTransform()[1],RasterLayer.GetGeoTransform()[5]))

We can also use GDAL to get the projection information. The PCRaster format doesn't store the projection information, but when we converted the rasters it saves an XML file with the projection info. The GDAL function is `GetProjection()`.

In [ ]:
RasterLayer.GetProjection()

That gives us a string in the OGC WKT format.
In order to make the projection and the units readable, we need to parse the OGC WKT string. For that we'll use the [PyCRS](https://pypi.org/project/PyCRS/) library. 

If it's not installed yet if you run this notebook locally, you can install the library by typing the following at the Anaconda prompt:

`conda install -c conda-forge pycrs`

First we import the library and save the OGC WKT string in a variable that we parse.

In [ ]:
import pycrs
RasterLayerProjection = RasterLayer.GetProjection()

# Parse OGC WKT string
crs = pycrs.parse.from_ogc_wkt(RasterLayerProjection)

We can check if it is a projected coordinate system:

In [ ]:
isinstance(crs, pycrs.ProjCS)

The result is `True` so we're not dealing with a Geographical Coordinate System.
Let's get the name of the projection.

In [ ]:
ProjectionName = crs.name
print(ProjectionName)

We can also get the units of the projection:

In [ ]:
ProjectionUnits = crs.unit.unitname.ogc_wkt
print(ProjectionUnits)

Finally it would be useful to get some statistics from the raster layer. Let's calculate the minimum and maximum value.
Because raster layers can have multiple bands we need to select the band. In our case we have a single band raster layer, so we have to choose band 1.

In [ ]:
RasterLayerBand = RasterLayer.GetRasterBand(1)

Now we can calculate the minumum and maximum value of the band using respectively `GetMinimum` and `GetMaximum`.

In [ ]:
RasterLayerMinimum = RasterLayerBand.GetMinimum()
RasterLayerMaximum = RasterLayerBand.GetMaximum()
print("Minimum: ", RasterLayerMinimum)
print("Maximum: ", RasterLayerMaximum)

Let's write this in a function and get the raster layer properties for all the raster layers so we can compare.

In [ ]:
from osgeo import gdal
import pycrs

def RasterLayerProperties(RasterLayer):
    print("Raster file: {}".format(RasterLayer.GetDescription()))
    print("Driver: {}/{}".format(RasterLayer.GetDriver().ShortName,
                            RasterLayer.GetDriver().LongName))
    print("Size is {} x {} x {}".format(RasterLayer.RasterXSize,
                                    RasterLayer.RasterYSize,
                                    RasterLayer.RasterCount))
    RasterLayerProjection = RasterLayer.GetProjection()
    crs = pycrs.parse.from_ogc_wkt(RasterLayerProjection)
    print("Projection:",crs.name)
    print("Map units:",crs.unit.unitname.ogc_wkt)
    geotransform = RasterLayer.GetGeoTransform()
    if geotransform:
        print("Origin = ({}, {})".format(geotransform[0], geotransform[3]))
        print("Pixel Size = ({} {}, {} {})".format(geotransform[1],crs.unit.unitname.ogc_wkt, \
                                                   geotransform[5],crs.unit.unitname.ogc_wkt))
    RasterLayerBand = RasterLayer.GetRasterBand(1)
    print("Minimum: {}".format(RasterLayerBand.GetMinimum()))
    print("Maximum: {}".format(RasterLayerBand.GetMaximum()))
    
    print()
    RasterLayer = None
    
    
DTMLayer = gdal.Open( "data/dtm.map" )
RasterLayerProperties(DTMLayer)

BuildgLayer = gdal.Open( "data/buildg.map" )
RasterLayerProperties(BuildgLayer)

RoadsLayer = gdal.Open( "data/roads.map" )
RasterLayerProperties(RoadsLayer)

GWLevelLayer = gdal.Open( "data/gwlevel.map" )
RasterLayerProperties(GWLevelLayer)

Based on these results we can conclude that the raster layers have the same properties and are suitable for map algebra.

# 4. Condition 1: Wells within 150 Meters of Houses or Roads<a name="condition_1"></a>
## 4.1 Introduction

Now that we've converted the GeoTIFF files to PCRaster format and have checked their properties, it's time to proceed with analysing the three conditions:

* Condition 1: Wells within 150 meters of houses or roads
* Condition 2: No industry, mine, or landfill within 300 meters from wells
* Condition 3: Wells less than 40 meters deep

For Condition 1 we'll first look at the houses and we'll do the following:

1. Create a boolean layer with True for houses and False for other buildings
2. Create zones of 150 meters around the houses

Then we'll repeat the steps for roads.

## 4.2 Create a Boolean Layer with True for Houses and False for Other Buildings
We usually type the commands at the Python prompt. In this Jupyter Notebook we simulate this. I.e. everything you type in the fields is equal to a command at the Python prompt.

Let's first load the `pcraster` module.

In [ ]:
from pcraster import *

Remember that our data is stored in the `2j_Data` folder, while our Python is running in the parent folder. For convenience it's useful to change to the Data folder so all our input and output files are read and written there respectively.

With the `os` library we can use command line command.

In [ ]:
import os

Let's check the current working directory.

In [ ]:
print(os.getcwd())

We can use an equivalent of the `cd` command to change the directory: `os.chdir(stringWithPath)`, where `stringWithPath` defines the path.

In [ ]:
os.chdir("./2j_Data")

Check where we are now. Make sure that we are in the `2j_Data` folder of this tutorial.

Now we're ready to read `buildg.map` from disk and use it for map algebra with PCRaster.
We can read maps using `readmap(stringWithPathToFile)`.

In [ ]:
Buildings = readmap("buildg.map")

Now we have read `buildg.map` from disk and can refer to it with the variable `Buildings`.
There are different ways to visualise the map:
* In Python we can use the `aguila` operator: `aguila(Buildings)`
* At the command prompt we can use the `Aguila` command with the file name: `aguila buildg.map`
* Use Python plotting tools that also work with Jupyter notebooks on Binder etc.

We're going to use the last option in this Jupyter notebook. But you can try the other ones too if you run this notebook locally.

We can use the `plot` operator to visualise PCRaster maps. It has the following syntax:

```Python
plot(raster, labels=None, title=None, filename=None)
```

The inputs are:

`raster`: Raster map with type of either Boolean, Nominal, Scalar, or Ldd.

`labels`: Optional. Dictionary of labels that should be used for the legend, cell values will be used otherwise.

`title`: Optional. Legend title as string, tries to identify the variable name otherwise.

`filename`: Optional. If provided plot will be written to disk.

Raster data has no attribute table. The values in the `Buildings` raster represent the following classes:

| Value | Class |
|:-------:|:-------:|
| 0 | None |
| 1 | House |
| 2 | Public building |
| 3 | Landfill |
| 4 | Industry |
| 5 | Mine |

We can save the legend in a Python dictionary:

In [ ]:
Legend = {0:"None",1:"House",2:"Public building",3:"Landfill",4:"Industry",5:"Mine"}

We can now plot the Buildings layer by executing this code:

In [ ]:
plot(Buildings, labels=Legend, title="Buildings", filename=None)

We first need to create a boolean layer with True for houses and False for other buildings.

In [ ]:
Houses = Buildings == 1

Here we have used the `==` operator. This creates a boolean map with value 1 for all cells that are equal to 1 and 0 for all other cells. Here you can find more info about [operators in PCRaster](https://pcraster.geo.uu.nl/pcraster/latest/documentation/python/quickstart.html#operators).
Let's visualise the result with the `plot` operator:

## 4.3 Create Zones of 150 Meters Around the Houses
The next step is to create zones of 150 m around the houses.

We can use the [`spreadmaxzone` function](https://pcraster.geo.uu.nl/pcraster/latest/documentation/pcraster_manual/sphinx/op_spreadmaxzone.html).

The syntax is `Result = spreadmaxzone(points, initialfrictiondist, friction, max_distance)`
In our case `points` are the `Houses`, `initialfrictiondist = 0`, `friction = 1` and `max_distance = 150`. `intialfrictiondistance = 0` means that their is no friction at the location of the houses an with `friction = 1` we calculate the distance from other pixels in an equal way without different weights.

In [ ]:
houses150m = spreadmaxzone(Houses, 0, 1, 150)

Visualise the result.

We can save this result to disk using the `report` function. The first argument is the variable name of the map that you want to write to disk. The second argument is the string of the file name.

In [ ]:
report(houses150m,"houses150m.map")

If you run this notebook locally, verify that `houses150m.map` is stored in the `2j_Data` folder and visualise the result by using aguila from the command prompt:
```
aguila houses150.map
```

Alternatively, you can run terminal commands from a notebook by adding `!` to the command.

In [ ]:
!aguila houses150.map

## 4.4 Create Zones of 150 Meters Around the Roads
In a similar way we can now calculate the 150 m buffer around the roads.

The values in `roads.map` represent the following classes:

| Value | Class |
|:-------:|:-------:|
| 0 | No road |
| 1 | Dirt road |
| 2 | Tarmac |

Do the same steps as for `houses150m` and save the result to `roads150m.map`.

Visualise the result.

Save the result to disk with the name `roads150m.map`.

# 5. Condition 2: No Industry, Mine, or Landfill within 300 Meters from Wells<a name="condition_2"></a>
## 5.1 Introduction
For the second condition we first need to reclassify the `buildg` layer in such a way that the result is a Boolean map with True (1) for industry, mine, and landfill and False (0) for the other classes. Then we need to calculate the distance to the True pixels and finally calculate the pixels that are further than 300 m from industry, mine, and landfill.

## 5.2 Create a Boolean Raster with True for Industry, Mine, and Landfill, and False for Other Buildings

Let's first reclassify `buildg.map`. For reclassifications we use so called *Lookup tables*.
These lookup tables can be used by PCRaster witht the [`lookup...` operations](https://pcraster.geo.uu.nl/pcraster/latest/documentation/pcraster_manual/sphinx/op_lookup.html). For these operations you need to know the data type of the output and use one of these operations:

| output data type | operation |
|:-------:|:-------:|
| boolean | `lookupboolean` |
| nominal | `lookupnominal` |
| ordinal | `lookupordinal` |
| scalar | `lookupscalar` |
| directional | `lookupdirectional` |
| ldd | `lookupldd` |

The general syntax is:
`Result = lookup<datatype>(lookuptable,input map)`

In our case we need to reclassify `buildg.map` to a boolean result, so the operation is `lookupboolean`.

In this case the table should be a text file with the following contents:
```
0 0
1 0
2 0
3 1
4 1
5 1
```

You can prepare this text file in notepad or another plain text editor. You can give it any extension, but here we'll use `.tbl` so it's clear that it's a table. Here we'll use the one provided in the `2j_Data` folder: `industry.tbl`.

We can also reclassify ranges of values and use multiple input maps. We will look at that later.

Now we can use the `lookupboolean` operation.

In [ ]:
Industry = lookupboolean("industry.tbl", Buildings)

Visualise the result.

## 5.3 Create Zones of 300 Meters Around Industry, Mine, and Landfill
Now we have the industry pixels we can create a boolean map with True for pixels that are at least 300 m from industry and False for pixels that are closer than 300 m from industry.

Previously we used the `spreadmaxzone` operation. There is, however, no `spreadminzone` operation.
There are different ways to solve this:
1. We can use `spreadmaxzone` and invert the result by using the NOT operator `~`.
2. We can calculate all distances with the `spread` operation and use the `>=` operator to create the required boolean map.
3. We can use a lookup table reclassifying the ranges.

Method 1 is the fastest. Let's try that first.

In [ ]:
IndustryMax300m = spreadmaxzone(Industry, 0, 1, 300)
IndustryMin300m = ~ IndustryMax300m

Visualise the result.

Let's look also at the other methods and see if they give the same result.

Calculate the distances to Industry:

In [ ]:
IndustryDistance = spread(Industry,0,1)

Create the boolean condition:

In [ ]:
IndustryMin300m_2 = IndustryDistance >= 300

Visualise the result.

For the third method we're going to use a lookup table with ranges. In the text file the first column containst the ranges and the second column the output values. `[` and `]` are used to include the value in the range and `<` and `>` are used to exclude the value in the range.

So here our text file looks like:

```
<,300> 0
[300,> 1
```

Where the first row means: all values < 300 get boolean zero. The second row means all values >= 300 get boolean 1.

We're going to reclassify `IndustryDistance` using the lookup table `industry300m.tbl` which is in the `2j_Data` folder. Write the code below and write the result to `IndustryMin300m_3`:

Visualise the result.

You can see that the results are the same.
In GIS you often get to the same result in different ways. You need to find the most efficient route to the desired result.

Save the result as `ind300m.map`.


# 6.1 Condition 3: Wells Less than 40 Meters Deep<a name="condition_3"></a>
For the last condition we need to identify the wells that are less than 40 m deep.

`gwlevel.map` gives the absolute elevation of the groundwater level in the well in meters above sea level. In order to calculate the depth to the groundwater, we need to subtract this from the surface elevation given in the digital terrain model (`dtm.map`).

Read `gwlevel.map` and `dtm.map` from disk and use the variables `GWLevel` and `DTM` respectively. Try to do that by yourself in the next line.

Now we can calculate the well depth by subtracting the rasters:

In [ ]:
WellDepth = DTM - GWLevel

Let's visualise the result. Is it boolean, nominal or scalar?

We can calculate a boolean raster where pixels with a well depth < 40 are True and >= 40 are False. Type the code in the next field and write the output to variable `NotDeep`.

Visualise and save the result to `notdeep.map`.

# 7. Combine the Three Conditions<a name="combine"></a>
After calculating the Boolean maps for the three conditions we need to combine them to come to the final result.

The result should be a boolean map where accessible wells are located at cells where all conditions are True. Other cells should be False. Which boolean operator should we use?

A. OR

B. NOT

C. AND

D. XOR

Type the code in the field below. Use the correct [operator](https://pcraster.geo.uu.nl/pcraster/latest/documentation/python/quickstart.html#operators). Write the results to variable `AccessibleWells`.

Visualise the result and save it disk as `accessiblewells.map`.

# 8. What is the advantage of scripting map algebra?<a name="conclusion"></a>
The same procedure could be easily done in QGIS as you can see in [this tutorial](https://courses.gisopencourseware.org/mod/book/view.php?id=68). So what are then the advantages of scripting this instead of using a graphical user interface (GUI)?

1. You have more control over the functions, because you determine each argument and operator that you use.
2. The process becomes reproducable. Anyone else can run the script with the input data to obtain the same results. The code is transparent (open source).
3. You can easily modify the critera and see how it affects the result.

Therefore we're going to put all code that we used in this tutorial into one script, where certain processes are written as functions. You can try it first yourself. Below you'll find the answer.

In [ ]:
# Import the library
from pcraster import *

def WithinDistance(raster,maxdistance):
    ResultWithinDistance = spreadmaxzone(raster, 0, 1, maxdistance)
    return ResultWithinDistance
    
def BeyondDistance(raster,mindistance):
    ResultBeyondDistance = ~ spreadmaxzone(raster, 0, 1, mindistance)
    return ResultBeyondDistance

# Change to data folder if needed
os.chdir("./data")

# Read all input maps
Buildings = readmap("buildg.map")
Roads = readmap("roads.map")
GWLevel = readmap("gwlevel.map")
DTM = readmap("dtm.map")

# Set thresholds for conditions
DistanceCondition1 = 150
DistanceCondition2 = 300
DepthCondition3 = 40

# Condition 1: Wells within X Meters of Houses or Roads
Houses = Buildings == 1
Houses150m = WithinDistance(Houses,DistanceCondition1)

IsRoad = Roads != 0
Roads150m = WithinDistance(IsRoad,DistanceCondition1)

# Condition 2: No Industry, Mine, or Landfill within X Meters from Wells
Industry = lookupboolean("industry.tbl", Buildings)
IndustryMin300m = BeyondDistance(Industry,DistanceCondition2)

# Condition 3: Wells Less than X Meters Deep
WellDepth = DTM - GWLevel
NotDeep = WellDepth < DepthCondition3

# Combine conditions
AccessibleWells = Houses150m & Roads150m & IndustryMin300m & NotDeep

# Write result to disk and visualise
report(AccessibleWells,"accessiblewells.map")
plot(AccessibleWells,labels={0:"Not accessible",1:"Accessible"},title="Wells",filename=None)

Now you can play with the values for the conditions and check the results.
What happens if we change the distance to houses and roads to 250 m?